In [1]:
import os
from pathlib import Path
# from matplotlib import pyplot as plt
# from matplotlib import cm
import pandas as pd
import yaml
# import glob
# import seaborn as sns
# from Bio import SeqIO

# Mitochondria phylogeny

In [2]:
proj_dir="/master/nplatt/sch_hae_its-nigeria"
results_dir=f"{proj_dir}/results"

In [3]:
Path(f"{results_dir}/mito_phylogeny").mkdir(parents=True, exist_ok=True)
os.chdir(f"{results_dir}/mito_phylogeny")

In [4]:
info_df=pd.read_csv(f"{proj_dir}/its-nigeria_samplesheet.csv", sep=",")
info_df

,wgs_id,its_id,sra,species,country,locale,patient,miracidum
0,Sb_NG_ak_1.1,Sb.ng.ak.1.1F,NaN,sbovis,nigeria,ak,1,1
1,Sb_NG_ak_2.1,sb_ng_ak_2.1,NaN,sbovis,nigeria,ak,2,1
2,Sb_NG_ak_2.2,Sb.ng.ak.2.2F,NaN,sbovis,nigeria,ak,2,2
3,Sb_NG_ak_2.3,Sb.ng.ak.2.3F,NaN,sbovis,nigeria,ak,2,3
4,Sb_NG_ak_3.1,Sb.ng.ak.3.1R,NaN,sbovis,nigeria,ak,3,1
...,...,...,...,...,...,...,...,...
200,Sh_NG_os_3_1,sh_ng_os_3_1,NaN,shaematobium,nigeria,osun,3,1
201,c_Sh_NG_os_3_11,Sh.ng.os.3.11F,NaN,shaematobium,nigeria,osun,3,11
202,c_Sh_NG_os_3_5,Sh.ng.os.3.5F,NaN,shaematobium,nigeria,osun,3,5
203,c_Sh_NG_os_3_6,Sh.ng.os.3.6F,NaN,shaematobium,nigeria,osun,3,6


In [ ]:
%%bash

conda run -n mito_assembly --live-stream clustalo \
    -INFILE=../mito_assembly/mito_assemblies.fasta \
    -OUTFILE=mito_assemblies.aligned.fasta \
    -OUTPUT=FASTA

In [8]:
%%bash

conda run -n trimal trimal -in mito_assemblies.aligned.fasta -out mito_assemblies.aligned.trimmed.fasta -automated1

In [ ]:
%%bash 

conda run -n raxml --live-stream \
    raxml-ng \
        --all \
        --msa mito_assemblies.aligned.trimmed.fasta \
        --msa-format FASTA \
        --model GTR \
        --tree pars{50},rand{50} \
        --bs-trees 1000 \
        --redo

In [13]:
!cat mito_assemblies.aligned.trimmed.fasta.raxml.log


RAxML-NG v. 1.2.2 released on 11.04.2024 by The Exelixis Lab.
Developed by: Alexey M. Kozlov and Alexandros Stamatakis.
Contributors: Diego Darriba, Tomas Flouri, Benoit Morel, Sarah Lutteropp, Ben Bettisworth, Julia Haag, Anastasis Togkousidis.
Latest version: https://github.com/amkozlov/raxml-ng
Questions/problems/suggestions? Please visit: https://groups.google.com/forum/#!forum/raxml

System: Intel(R) Xeon(R) Gold 6262V CPU @ 1.90GHz, 96 cores, 1007 GB RAM

RAxML-NG was called at 09-Oct-2025 16:24:16 as follows:

raxml-ng --all --msa mito_assemblies.aligned.trimmed.fasta --msa-format FASTA --model GTR --tree pars{50},rand{50} --bs-trees 1000 --redo

Analysis options:
  run mode: ML tree search + bootstrapping (Felsenstein Bootstrap)
  start tree(s): random (50) + parsimony (50)
  bootstrap replicates: parsimony (1000)
  random seed: 1760045056
  tip-inner: OFF
  pattern compression: ON
  per-rate scalers: OFF
  site repeats: ON
  logLH epsilon: general: 10.000000, brlen-triplet: 1

In [14]:
%%bash
conda run -n nwed nw_ed mito_assemblies.aligned.trimmed.fasta.raxml.support 'i & b<=50' o > mito_assemblies.aligned.trimmed.fasta.raxml.support.bs50.nwk

In [ ]:
%%bash
cat ../combined/merged.csv | sed 's/,/\t/g' | cut -f1,3,4,5,17,19,22 >annotations.tsv